# En esta libreta se incluy el código para hacer la descarga de la información de parte de la SEP con respecto a la matricula escolar por ciclo escolar. Para esto se descarga el Formato 911.  

## el Formato 911 es el principal instrumento de recolección de datos del sistema educativo. Su propósito es obtener información estadística confiable y actualizada sobre los principales componentes de la educación en México.

## ¿Qué es? 
### Es una serie de cuestionarios estadísticos que se aplican al inicio y al final de cada ciclo escolar.

## ¿Quién lo llena? 
### El director o responsable de cada centro de trabajo (es decir, cada escuela), tanto pública como privada, en todos los niveles educativos.

## ¿Qué información recaba? 
### Principalmente datos sobre alumnos (inscripción, existencia, por grado, edad y sexo), personal docente (cantidad, tipo de plaza), y la infraestructura de las escuelas.

#### La información se obtiene por medio de la Plataforma Nacional de Datos Abiertos. (www.datos.gob.mx)

In [6]:
import requests 
import os
import pandas as pd

In [3]:
# Creamos un diccionario con los ciclos escolares y sus URLs correspondientes.
# Esto hace que el código sea muy fácil de leer y de actualizar en el futuro.
archivos_a_descargar = {
    '2019-2020': 'https://repodatos.atdt.gob.mx/s_educacion_publica/f911/BASICA_2019-2020.csv',
    '2020-2021': 'https://repodatos.atdt.gob.mx/s_educacion_publica/f911/BASICA_2020-2021.csv',
    '2021-2022': 'https://repodatos.atdt.gob.mx/s_educacion_publica/f911/BASICA_2021-2022.csv',
    '2022-2023': 'https://repodatos.atdt.gob.mx/s_educacion_publica/f911/BASICA_2022-2023.csv',
    '2023-2024': 'https://repodatos.atdt.gob.mx/s_educacion_publica/f911/ESTANDAR_BASICA_I2324.csv' #ciclo 2023-2024
}

# Definimos la ruta de destino usando la ruta relativa.
ruta = '../data/raw/formato_911/'
os.makedirs(ruta, exist_ok=True) # Creamos la carpeta si no existe

print("--- Iniciando descarga de archivos del Formato 911 ---")

for ciclo, url in archivos_a_descargar.items():
    
    # Creamos un nombre de archivo estandarizado y limpio.
    nombre_archivo = f'formato_911_basica_{ciclo}.csv'
    ruta_guardado = os.path.join(ruta, nombre_archivo)
    
    # Verificamos si el archivo ya existe para no descargarlo de nuevo.
    if not os.path.exists(ruta_guardado):
        print(f"Descargando datos para el ciclo {ciclo}...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status() # Lanza un error si la descarga falla
            
            # Guardamos el archivo en disco
            with open(ruta_guardado, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            print(f" -> ✅ Archivo guardado en: {ruta_guardado}")

        except requests.exceptions.RequestException as e:
            print(f" -> ❌ Error al descargar el archivo para el ciclo {ciclo}: {e}")
    else:
        print(f"El archivo para el ciclo {ciclo} ya existe. Se omite.")

print("\n--- 🎉 ¡Proceso de descarga finalizado! ---")

--- Iniciando descarga de archivos del Formato 911 ---
Descargando datos para el ciclo 2019-2020...
 -> ✅ Archivo guardado en: ../data/raw/formato_911/formato_911_basica_2019-2020.csv
Descargando datos para el ciclo 2020-2021...
 -> ✅ Archivo guardado en: ../data/raw/formato_911/formato_911_basica_2020-2021.csv
Descargando datos para el ciclo 2021-2022...
 -> ✅ Archivo guardado en: ../data/raw/formato_911/formato_911_basica_2021-2022.csv
Descargando datos para el ciclo 2022-2023...
 -> ✅ Archivo guardado en: ../data/raw/formato_911/formato_911_basica_2022-2023.csv
Descargando datos para el ciclo 2023-2024...
 -> ✅ Archivo guardado en: ../data/raw/formato_911/formato_911_basica_2023-2024.csv

--- 🎉 ¡Proceso de descarga finalizado! ---


### También se hace la descarga en la misma plataforma el catálogo con el listado de centros educativos de la Secretaría de Educación Pública del estado de Sonora.

In [21]:
url_catalogo_escuelas = 'https://www.datos.gob.mx/dataset/2a1d047c-546b-4293-971a-c835689a37a5/resource/4f013342-5028-447f-b39d-1c08f09f47f3/download/catalogo_centro_trabajo_26_csv.csv'
ruta_raw = '../data/raw/'
nombre_archivo = 'catalogo_escuelas_sonora.csv'
ruta_guardado = os.path.join(ruta_raw, nombre_archivo)

# Creamos la carpeta si no existe
os.makedirs(ruta_raw, exist_ok=True)

print("--- Descargando Catálogo de Centros de Trabajo (Escuelas) de Sonora ---")

# 2. Verificamos si el archivo ya existe para no descargarlo de nuevo
if not os.path.exists(ruta_guardado):
    try:
        # 3. Usamos pandas para leer el CSV directamente desde la URL
        # OJO: Los archivos de gobierno a veces usan codificación 'latin1'
        df_catalogo = pd.read_csv(url_catalogo_escuelas, encoding='latin1', low_memory=False)

        # 4. Guardamos una copia local en la carpeta data/raw
        df_catalogo.to_csv(ruta_guardado, index=False, encoding='utf-8')

        print(f"✅ Catálogo de escuelas guardado exitosamente en: {ruta_guardado}")
        print("\nVista previa de las primeras 5 escuelas:")
        print(df_catalogo.head())

    except Exception as e:
        print(f"❌ Ocurrió un error al descargar o procesar el archivo: {e}")
else:
    print(f"El archivo '{nombre_archivo}' ya existe en data/raw/. Se omite la descarga.")

--- Descargando Catálogo de Centros de Trabajo (Escuelas) de Sonora ---
✅ Catálogo de escuelas guardado exitosamente en: ../data/raw/catalogo_escuelas_sonora.csv

Vista previa de las primeras 5 escuelas:
       cv_cct                                      c_nombre  cv_tipo  \
0  26BBE2696K  BIBLIOTECA DE LA PREPARATORIA POPULAR MEXICO        7   
1  26KPR0173F                             CURSO COMUNITARIO        9   
2  26KPR0193T                             CURSO COMUNITARIO        9   
3  26KPR0740I                             CURSO COMUNITARIO        9   
4  26PBH0080B        UNIVERSIDAD DEL DESARROLLO PROFESIONAL        9   

       c_tipo  cv_administrativa c_administrativa  cv_estatus c_estatus  \
0  BIBLIOTECA                 26           SONORA           0  INACTIVO   
1     ESCUELA                 26           SONORA           0  INACTIVO   
2     ESCUELA                 26           SONORA           0  INACTIVO   
3     ESCUELA                 26           SONORA           1  